In [ ]:
import os
import json
import time
import re
import pandas as pd
import numpy as np
from datetime import datetime

from src.util import prnt
from src.util import print_all_output
from src.settings import *

# Step 1: Load

In [ ]:
%reload_ext autoreload
%autoreload 2

from src.load import Load
semester = 'WS2025_26' # WS2024_25 WS2023_24',   'dump20240826'), 'SS2025' 
# Step 1
l = Load(semester=semester, dump="dump20251115")
#l.process_textedits(filter_weeks=1)
#
l.run(filter_weeks=1)
df = l.get_data()
#df
# load all data of a group
#l.run(filter_group=[2751],filter_weeks=True)
df[df['moodle_group_id'] == 5083]

# Step 2: EasySync

In [ ]:
!pipx install numpy>=1.24.0 scikit-learn>=1.3.0
!pipx install textstat
!pipx install language_tool_python
!pipx install textdescriptives
!pipx install matplotlib
#!spacy download de_core_news_sm


In [ ]:
%reload_ext autoreload
%autoreload 2

from src.extract_easy_sync import Extract_Easy_Sync
from src.text_quality import Preprocess_Text_Quality
semester = 'WS2025_26'
period_split_interval='weeks'
# Step 2
#print_all_output = False
ptq = Preprocess_Text_Quality(semester)
sft = semester_form_to_arr[semester]  # variable in settings.py
time_breaks = ptq.split_text_progression_by_threshold(
    df,
    semester, 
    start_date=datetime(sft[0][0], sft[0][1], sft[0][2]), 
    end_date=datetime(sft[1][0], sft[1][1], sft[1][2]),
    group_column='moodle_pad_id',
    period_split_interval=period_split_interval
    )
time_breaks.sort_values(by=['moodle_group_id', 'moodle_pad_id'])

es = Extract_Easy_Sync(semester, time_breaks)
es.is_reconstructing_text = False
df_textchanges = es.extract_easy_sync(df)
df_textchanges


# [experimental] Step 2.1: Extract sessions

In [ ]:

from src.extract_sessions import Extract_Sessions

semester = 'SS2024'
es = Extract_Sessions(semester)
#dd = es.load_and_combine_df() # TODO: All authorids in the pad_commit file are derived from moodle but not from etherpad
#dd['authorid'].unique()
#dd[dd['authorid'].isnull()]
df_textchanges.columns
#df_textchanges['moodle_user_id'] = df_textchanges['moodle_user_id'].astype('int')

df_textchanges['moodle_group_id'] = df_textchanges['moodle_group_id'].astype('int')
df_sessions = es.extract_sessions(df_textchanges)
df_sessions



#  [experimental] Step 2.2: Extract and Analyze Text

In [ ]:
%reload_ext autoreload
%autoreload 2

"""
This script extracts all texts per group/pad and defined periods.
The resulting text versions are stored as files. Then the text is analyzed for certain features.
"""

import pandas as pd
import numpy as np
from datetime import datetime

from src.util import prnt
from src.util import print_all_output
from src.settings import *
from src.text_quality import Preprocess_Text_Quality
from src.extract_easy_sync import Extract_Easy_Sync

semester = 'SS2025'
period_split_interval='weeks'

# Step 1: identify time breaks
ptq = Preprocess_Text_Quality(semester)
sft = semester_form_to_arr[semester]  # variable in settings.py
time_breaks = ptq.split_text_progression_by_threshold(
    #df[df['moodle_group_id']==3925], 
    df,
    semester, 
    start_date=datetime(sft[0][0], sft[0][1], sft[0][2]), 
    end_date=datetime(sft[1][0], sft[1][1], sft[1][2]),
    group_column='moodle_pad_id',
    period_split_interval=period_split_interval
    )
time_breaks.sort_values(by=['moodle_group_id', 'moodle_pad_id'])


es = Extract_Easy_Sync(semester, time_breaks=time_breaks, period_split_interval=period_split_interval)
es.is_reconstructing_text = True
print_all_output = True

##pads = df[df['moodle_pad_id']==df['moodle_pad_id'].unique()[14]].groupby('moodle_pad_id', group_keys=True)
pads = df.groupby('moodle_pad_id', group_keys=True)

i = 0
for index, pad in pads:
    print(' ')
    for row in pad.sort_values(by=['timestamp']).itertuples():
        i = i + 1
        if i > 0:
            es.extract_changeset(
                row[5],             # row['textedit_changeset'], 
                timestamp=row[6],   # row['timestamp'], 
                group_id=row[3],    # row['moodle_group_id'], 
                pad_id=row[4],      # str(row['moodle_pad_id'])
            )
    #print(es.ttext)

ptq = Preprocess_Text_Quality(semester)
res_tq = ptq.determine_text_quality_from_files(period_split_interval=period_split_interval)
res_tq



###  [experimental] nlp test

In [ ]:
from src.text_quality import Preprocess_Text_Quality, Text_Quality
from src.extract_easy_sync import Extract_Easy_Sync
import de_core_news_sm

text = "Der Mann geht Joggen."
tq = Text_Quality()
tq.run(text, model='de_core_news_lg') # 'de_core_news_md'

import de_core_news_sm

nlp = de_core_news_sm.load()

tokenized = nlp(text)
for token in tokenized:
    print(token, token.pos_, token.ent_type_)


# Step 3 : Extract_Neighbours

In [ ]:
%reload_ext autoreload
%autoreload 2
from src.extract_neighbours import Extract_Neighbours

# Step 3
print_all_output=False
semester = 'SS2025'
en = Extract_Neighbours(semester)
author_relations = en.extract_neighbours(df_textchanges)
author_relations

#author_relations[author_relations['right_neighbor']>0]
#df_textchanges['moodle_pad_id_'] = df_textchanges['moodle_pad_id'].str.split('$', expand=True)[0]
#pads = df_textchanges['moodle_pad_id_'].unique()
#df_textchanges[df_textchanges['moodle_pad_id_'] == pads[3]]


# Step 4: Extract_Degree

In [ ]:
%reload_ext autoreload
%autoreload 2

from src.extract_degrees import Extract_Degree

# Step 4
semester = 'SS2025'
ed = Extract_Degree(semester)
author_relations_summary = ed.summarize_individual_level(author_relations)

#author_relations_summary
#author_relations_summary['right2'] = author_relations_summary['right'].astype('int').abs() 
#author_relations_summary['right2']
#author_relations_summary.dtypes

print_all_output = False #True
author_degrees = ed.extract_degree(author_relations_summary)
# TODO: During the processing the size of the dataframe varys pretty much. Something is wrong with the agg function. Check if this behavior is ok.
#print(author_relations_summary.size)
#print(author_degrees.size)
#author_degrees

print_all_output = True
author_degrees_per_group = ed.map_to_group(df_textchanges, author_degrees)
#print('df_textchanges', df_textchanges)
#print('author_degrees', author_degrees)
#author_degrees_per_group[author_degrees_per_group['indegree_count']>0]
author_degrees_per_group


# Step 5: Coll Graph

In [ ]:
!pip install matplotlib

In [ ]:
%reload_ext autoreload
%autoreload 2

from src.collaboration_graph import Collaboration_Graph

semester = 'WS2025_26'
cg = Collaboration_Graph(semester)
#cg.check_random_group(author_relations)
graph_measures = cg.create_graph_for_all_groups(author_relations, save_plot=False, save_output=True, show_plot=False)
graph_measures
cg.create_json_graph_for_all_groups(
    author_relations, 
    last_modified = 1763370511517,
    save_to_file=True,
    target_week="week-2"
    )


## Peer Review Data

In [ ]:
%reload_ext autoreload
%autoreload 2
from src.extract_peer_review import extract_peer_review

extract_peer_review(input_dir = './data/peer-review-dumps', output_dir = './output')

# All at once: Group behavior in all semesters


In [ ]:
!pipx install textstat==0.7.7

In [ ]:
%reload_ext autoreload
%autoreload 2

# Kernel: Python 3.12.5

"""Step 0: Run over all semesters"""
import gc  # Import garbage collector
import pandas as pd
import numpy as np
from datetime import datetime

from src.util import prnt
from src.util import print_all_output
from src.settings import *
from src.load import Load
from src.extract_easy_sync import Extract_Easy_Sync
from src.extract_sessions import Extract_Sessions
from src.extract_neighbours import Extract_Neighbours
from src.extract_degrees import Extract_Degree
from src.collaboration_graph import Collaboration_Graph
from src.text_quality import Preprocess_Text_Quality

semesters = [
    # not available # 'SS2021', 
    #('WS2021_22', 'dump20240826'),
    #('SS2022',    'dump20240826'),
    #('WS2022_23', 'dump20240826'),
    #('SS2023',    'dump20240826'),
    #('WS2023_24',   'dump20240826'),
    #('SS2024',    'dump20240826'),
    #('WS2024_25',  'dump20241209'),
    ('SS2025',   'dump20250520'),
    #('WS2025_26',  'dump20251215'),
]
project_name = 'edm26' # e.g., lak25
period_split_interval = 'weeks'
save_csv_output = True
print_all_output = False
compute_texts = False
for the_semester in semesters:
    print('# Process semester '+ the_semester[0])
    semester = the_semester[0]
    the_dump = the_semester[1]
    period_1 = get_period(semester, period_1_arr)
    period_2 = get_period(semester, period_2_arr)
    period_3 = get_period(semester, period_3_arr)
    # print('  Periods: ', period_1, period_2, period_3)
    print('## step1 - Load data ('+semester+')')
    l = Load(semester, the_dump)
    l.run(filter_weeks=1)
    #l.process_textedits(filter_weeks=1)
    df = l.get_data()
    
    # Step 2
    # Step 2.1: identify time breaks
    ptq = Preprocess_Text_Quality(semester)
    sft = semester_form_to_arr[semester]  # variable in settings.py
    time_breaks = ptq.split_text_progression_by_threshold(
        df,
        semester, 
        start_date=datetime(sft[0][0], sft[0][1], sft[0][2]), 
        end_date=datetime(sft[1][0], sft[1][1], sft[1][2]),
        group_column='moodle_pad_id',
        #group_column='moodle_group_id',
        period_split_interval=period_split_interval
        )
    time_breaks.sort_values(by=['moodle_group_id', 'moodle_pad_id'])
    
    print('## step2 - Extract_Easy_Sync ('+semester+')')
    es = Extract_Easy_Sync(semester, time_breaks)
    es.is_reconstructing_text = compute_texts
    df_textchanges = es.extract_easy_sync(df)
    del df  # Free memory
    gc.collect()  # Force garbage collection
    
    print('## step3 Extract Sessions (skipped) ('+semester+')') # TODO
    ##es = Extract_Sessions(semester)
    ##df_sessions = es.extract_sessions(df_textchanges)
    
    # Step 4
    print('## step4 - Extract Text Versions (optional) ('+semester+')')
    if compute_texts == True:
        all_text = pd.read_csv(f'{output_path}{project_name}-{semester}-02.1-text-revisions.csv')
        ptq = Preprocess_Text_Quality(semester)
        #sft = semester_form_to_arr[semester]  # variable in settings.py
        df_raw_text = ptq.split_text_progression_by_threshold(
            all_text, 
            semester, 
            datetime(sft[0][0], sft[0][1], sft[0][2]), 
            datetime(sft[1][0], sft[1][1], sft[1][2])
            )
        res = ptq.determine_text_quality(df_raw_text) # will be saved to file 2.2
    
    
    en = Extract_Neighbours(semester, period_split_interval, save_output=save_csv_output)
    
    ed = Extract_Degree(semester, period_split_interval, save_output=save_csv_output)
    df_textchanges['timestamp'] = df_textchanges['timestamp'].astype(float)
    date_thresholds = es.generate_observation_times(
        datetime(sft[0][0], sft[0][1], sft[0][2]), 
        datetime(sft[1][0], sft[1][1], sft[1][2]),
        period_split_interval=period_split_interval
        )
    date_thresholds = [1765752289451] # hotfix
    print(date_thresholds)
    for subset_until in date_thresholds:
        print(f'## process period {list(date_thresholds).index(subset_until)+1}/{len(date_thresholds)}')
        df_textchanges_until_break = df_textchanges[df_textchanges['timestamp'] < subset_until]
        if df_textchanges_until_break.empty:
            print('arg xxx')
            continue
        
        print('### step5.1 - Extract_Neighbours ('+semester+')')
        author_relations = en.extract_neighbours(df_textchanges_until_break, subset_until) # before: df_textchanges xxx
        
        print('### step5.2 - Extract Degree ('+semester+')')
        author_relations_summary = ed.summarize_individual_level(author_relations, subset_until)
        author_degrees = ed.extract_degree(author_relations_summary, subset_until)
        author_degrees_per_group = ed.map_to_group(df_textchanges_until_break, author_degrees, subset_until)  
        del df_textchanges_until_break, author_degrees_per_group, author_relations_summary
        gc.collect()
        
        print('### step6 - Collaboration_Graph ('+semester+')')
        cg = Collaboration_Graph(semester, period_split_interval, save_output=save_csv_output)
        # TODO: save_plot imply displaying the plot
        graph_measures = cg.create_graph_for_all_groups(author_relations, subset_until, save_plot=False, save_output=True, show_plot=False)
        
        #graph_measures
        cg.create_json_graph_for_all_groups(
            author_relations, 
            last_modified = 1765752289451,
            save_to_file=True,
            target_week="week-6"
            )
        del author_relations, author_degrees, cg, graph_measures
        gc.collect()
    # fin

    del df_textchanges
    gc.collect()


# [experimental] All at once: Text features for all text versions in all semesters



In [ ]:
%reload_ext autoreload
%autoreload 2

"""
This script iterates over all semesters and loads the textedit data from csv and extracts the EasySync data. 
During the extraction versions of the text documents are reconstructed for provided periods (e.g. per day or week). 
The resulting text version is stored as txt file in the output folder
"""

import gc  # Import garbage collector
import pandas as pd
import numpy as np
from datetime import datetime

from src.util import prnt
from src.util import print_all_output
from src.settings import *
from src.load import Load
from src.extract_easy_sync import Extract_Easy_Sync
from src.text_quality import Preprocess_Text_Quality

semesters = [
    # not available # 'SS2021', 
    ##('WS2021_22', 'dump20240826'),
    ##('SS2022',    'dump20240826'),
    ##('WS2022_23', 'dump20240826'),
    ##('SS2023',    'dump20240826'),
    ##('WS2023_24',   'dump20240826'),
    ##('SS2024',    'dump20240826'),
    ('WS2024_25',  'dump20241209'),
]
period_split_interval = 'weeks'
print_all_output = False
compute_texts = False
for the_semester in semesters:
    print('# Process semester '+ the_semester[0])
    semester = the_semester[0]
    the_dump = the_semester[1]
    period_1 = get_period(semester, period_1_arr)
    period_2 = get_period(semester, period_2_arr)
    period_3 = get_period(semester, period_3_arr)
    print('  Periods: ', period_1, period_2, period_3)
    print('## step1 - Load data')
    l = Load(semester, the_dump)
    #l.run()
    l.process_textedits(filter_weeks=1)
    df = l.get_data()

    print(f'## step2 - identify time breaks in semester {the_semester[0]}') 
    ptq = Preprocess_Text_Quality(semester)
    sft = semester_form_to_arr[semester]  # variable in settings.py
    time_breaks = ptq.split_text_progression_by_threshold(
        df,
        semester, 
        start_date=datetime(sft[0][0], sft[0][1], sft[0][2]), 
        end_date=datetime(sft[1][0], sft[1][1], sft[1][2]),
        group_column='moodle_pad_id',
        period_split_interval=period_split_interval
        )
    time_breaks.sort_values(by=['moodle_group_id', 'moodle_pad_id'])

    print(f'## step3 - extract text versions in semester {the_semester[0]}') 
    es = Extract_Easy_Sync(semester, time_breaks=time_breaks, period_split_interval=period_split_interval)
    es.is_reconstructing_text = True
    print_all_output = True
    pads = df.groupby('moodle_pad_id', group_keys=True)
    del df
    gc.collect()

    i = 0
    for index, pad in pads:
        print(' ')
        for row in pad.sort_values(by=['timestamp']).itertuples():
            i = i + 1
            if i > 0:
                es.extract_changeset(
                    row[5],             # row['textedit_changeset'], 
                    timestamp=row[6],   # row['timestamp'], 
                    group_id=row[3],    # row['moodle_group_id'], 
                    pad_id=row[4],      # str(row['moodle_pad_id'])
                )
        #print(es.ttext)
    del pads, es
    gc.collect()



In [ ]:
%reload_ext autoreload
%autoreload 2

"""
Compute text features from all text versions
"""
import gc  # Import garbage collector
from src.settings import *
from src.text_quality import Preprocess_Text_Quality

semesters = [
    # not available # 'SS2021', 
    ##('WS2021_22', 'dump20240826'),
    ('SS2022',    'dump20240826'),
    ##('WS2022_23', 'dump20240826'),
    ##('SS2023',    'dump20240826'),
    ##('WS2023_24',   'dump20240826'),
    ##('SS2024',    'dump20240826'),
    ##('WS2024_25',  'dump20241209'),
]
period_split_interval = 'weeks'
min_index = 3500
max_index = 5000
for the_semester in semesters:
    semester = the_semester[0]
    ptq = Preprocess_Text_Quality(semester)    
    print('## step4 - Compute text features') 
    prefix = 'lak25-' + semester
    res_tq = ptq.determine_text_quality_from_files(
        period_split_interval=period_split_interval, 
        prefix = prefix,
        min_index = min_index,
        max_index = max_index
        )
    del ptq.tq, ptq
    gc.collect()
res_tq

# Playground


In [ ]:
import pandas as pd
import numpy as np
import re
import os
from datetime import datetime

# Define paths
the_paths = [
    #'datasets/CSCL01-WS2019_20-M1-DivAdapt',
    #'datasets/CSCL02-SS2020-M1-DivAdapt'
    #'datasets/CSCL03-WS2020_21-M1-DivAdapt',
    #'datasets/CSCL04-SS2021-M1-DivAdapt',
    'datasets/CSCL05-WS2021_22-M1-DivAdapt',
    'datasets/CSCL06-SS2022-M1-DivAdapt',
    'datasets/CSCL07-WS2022_23-M1-DivAdapt',
    'datasets/CSCL08-SS2023-M1-DivAdapt',
    'datasets/CSCL09-WS2023_24-M1-MULTIDIVERSE-CSCL',
    'datasets/CSCL10-SS2024-M1-MULTIDIVERSE-CSCL',
    'datasets/CSCL11-WS2024_25-M1-MULTIDIVERSE-CSCL',
    'datasets/CSCL12-SS2025-M1-MULTIDIVERSE-CSCL'
]

# Process group data
df_list = []
for i, path in enumerate(the_paths):
    # Extract semester using regex
    semester_match = re.search(r"(SS|WS)\d{4}(_\d{2})?", path)
    semester = semester_match.group(0) if semester_match else ""
    
    print(f'Load data of {semester}')
    filepath = f'../../{path}/data/{semester}-etherpad-07-group-graph-measures-groups.csv'
    
    # Read CSV file
    tmp = pd.read_csv(filepath)
    
    # Convert 'until' to datetime and calculate weeks
    tmp['until'] = pd.to_datetime(tmp['until'])
    min_until = tmp['until'].min()
    tmp['week'] = ((tmp['until'] - min_until).dt.days // 7) + 1
    
    # Filter and select columns
    tmp = (tmp
           .query('week < 7')
           .query('activeUsers > 1')
           .drop(columns=['until']))
    
    df_list.append(tmp)

# Combine all dataframes
df_groups = pd.concat(df_list, ignore_index=True)

# Remove specific columns
columns_to_drop = ['density2', 'degree_centrality', 'closeness_centrality']
df_groups = df_groups.drop(columns=[col for col in columns_to_drop if col in df_groups.columns])

print("df_groups shape:", df_groups.shape)
print("df_groups columns:", df_groups.columns.tolist())

# Process member data
df_list_members = []
for i, path in enumerate(the_paths):
    # Extract semester using regex
    semester_match = re.search(r"(SS|WS)\d{4}(_\d{2})?", path)
    semester = semester_match.group(0) if semester_match else ""
    
    print(f'Load data of {semester}')
    filepath = f'../../{path}/data/{semester}-etherpad-06-author-degrees.csv'
    
    # Read CSV file
    tmp = pd.read_csv(filepath)
    
    # Convert 'until' to datetime and calculate weeks
    tmp['until'] = pd.to_datetime(tmp['until'])
    min_until = tmp['until'].min()
    tmp['week'] = ((tmp['until'] - min_until).dt.days // 7) + 1
    
    # Filter by week
    tmp = tmp.query('week < 7')
    
    # Group by moodle_group_id and week, then summarize
    tmp = (tmp
           .groupby(['moodle_group_id', 'week'])
           .agg({
               'moodle_user_id': 'count',  # group_size = n()
               'outdegree_chars': 'sum',
               'selfdegree_chars': 'sum', 
               'selfdegree_count': 'std'
           })
           .reset_index())
    
    # Rename and calculate columns
    tmp = tmp.rename(columns={
        'moodle_user_id': 'group_size',
        'selfdegree_count': 'selfdegree_sd'
    })
    
    # Calculate total_chars
    tmp['total_chars'] = tmp['outdegree_chars'] + tmp['selfdegree_chars']
    
    # Drop the individual char columns and keep only what we need
    tmp = tmp.drop(columns=['outdegree_chars', 'selfdegree_chars'])
    
    # Sort and add semester
    tmp = tmp.sort_values(['moodle_group_id', 'week'])
    tmp['semester'] = semester
    
    df_list_members.append(tmp)

# Combine all member dataframes
df_members = pd.concat(df_list_members, ignore_index=True)

print("df_members shape:", df_members.shape)
print("df_members columns:", df_members.columns.tolist())

# Join the dataframes (right join)
df = df_groups.merge(
    df_members.drop(columns=['semester', 'group_size']), 
    on=['moodle_group_id', 'week'], 
    how='right'
)

# Relocate semester column after week
cols = df.columns.tolist()
if 'semester' in cols:
    cols.remove('semester')
    week_idx = cols.index('week')
    cols.insert(week_idx + 1, 'semester')
    df = df[cols]

print("Final df shape:", df.shape)
print("Final df columns:", df.columns.tolist())
print("\nFirst few rows:")
print(df.head())

In [ ]:
# prep tslearn data

%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from tslearn.utils import to_time_series_dataset

# Step 1: Load the CSV file
csv_path = "/Users/nise/Documents/proj_001_doc/pub/110-2025-LAK-CollWriting/cscl-writing/data/projects/lak2026/output/df-cluster.csv"  # Replace with your actual file path
df = pd.read_csv(csv_path)

# Step 2: Define the feature columns (all except identifiers)
feature_columns = [
    "activeUsers", "isolatedUsers", "usersProvideLotHelp", 
    "usersProvideAvgHelp", "usersProvideLittleHelp", "usersProvideNoHelp",
    "usersReceiveLotHelp", "usersReceiveAvgHelp", "usersReceiveLittleHelp",
    "usersReceiveNoHelp", "closeCouples", "avgCouples", "loseCouples",
    "density1", 
    #"total_chars", 
    "selfdegree_sd"
]

# Step 3: Get unique group IDs
unique_groups = df['moodle_group_id'].unique()
print(f"Found {len(unique_groups)} unique moodle groups")

# Step 4: Create time series for each group
time_series_list = []
group_ids = []

for group_id in unique_groups:
    # Filter data for this group
    group_data = df[df['moodle_group_id'] == group_id].copy()
    
    # Sort by week to ensure proper time ordering
    group_data = group_data.sort_values('week')
    
    # Extract feature values for all weeks
    # Each row becomes a time point, each column becomes a feature dimension
    group_time_series = group_data[feature_columns].values
    
    # Only add groups that have data
    if len(group_time_series) > 0:
        time_series_list.append(group_time_series)
        group_ids.append(group_id)

# Step 5: Convert to tslearn format
formatted_dataset = to_time_series_dataset(time_series_list)
formatted_dataset = np.nan_to_num(formatted_dataset, nan=0.0)

print(f"Group IDs created: {len(group_ids)} groups")

# Step 6: Create labels (choose one of the options below)

# Option 1: Create labels based on a column in your data (e.g., semester)
# Get the semester for each group (using the first occurrence for each group)
y = []
for group_id in group_ids:
    group_semester = df[df['moodle_group_id'] == group_id]['week'].iloc[0]
    y.append(group_semester)
y = np.array(y)


# Step 7: Display information about the dataset
print(f"Dataset shape: {formatted_dataset.shape}")
print(f"Number of time series: {formatted_dataset.shape[0]}")
print(f"Number of time points per series: {formatted_dataset.shape[1]}")
print(f"Number of features per time point: {formatted_dataset.shape[2]}")
print(f"Labels shape: {y.shape}")
print(f"Unique labels: {np.unique(y)}")

# Step 8: Now you can use with tslearn
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
knn = KNeighborsTimeSeriesClassifier(n_neighbors=5)
knn.fit(formatted_dataset, y)

from tslearn.svm import TimeSeriesSVC
#clf = TimeSeriesSVC(C=1.0, kernel="gak")
#clf.fit(formatted_dataset, y)

from tslearn.shapelets import LearningShapelets
#clf = LearningShapelets(n_shapelets_per_size={4: 1})
#clf.fit(formatted_dataset, y)


In [ ]:
# tslearn timeseries clustering .. multivariate

from tslearn.clustering import TimeSeriesKMeans, KernelKMeans

# Method 1: Elbow method to find optimal number of clusters
def find_optimal_clusters_elbow(data, max_k=20):
    inertias = []
    K_range = range(2, max_k + 1)
    
    for k in K_range:
        kmeans = TimeSeriesKMeans(n_clusters=k, random_state=42)
        kmeans.fit(data)
        inertias.append(kmeans.inertia_)
    
    # Plot elbow curve
    plt.figure(figsize=(10, 6))
    plt.plot(K_range, inertias, 'bo-')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Inertia')
    plt.title('Elbow Method for Optimal k')
    plt.grid(True)
    plt.show()
    
    return K_range, inertias

# Method 2: Silhouette analysis
def find_optimal_clusters_silhouette(data, max_k=20):
    silhouette_scores = []
    K_range = range(2, max_k + 1)
    
    for k in K_range:
        kmeans = TimeSeriesKMeans(n_clusters=k, random_state=42)
        cluster_labels = kmeans.fit_predict(data)
        
        # Flatten the time series for silhouette calculation
        data_flat = data.reshape(data.shape[0], -1)
        silhouette_avg = silhouette_score(data_flat, cluster_labels)
        silhouette_scores.append(silhouette_avg)
        print(f"k={k}: Silhouette Score = {silhouette_avg:.3f}")
    
    # Plot silhouette scores
    plt.figure(figsize=(10, 6))
    plt.plot(K_range, silhouette_scores, 'ro-')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Average Silhouette Score')
    plt.title('Silhouette Analysis for Optimal k')
    plt.grid(True)
    plt.show()
    
    optimal_k = K_range[np.argmax(silhouette_scores)]
    print(f"Optimal number of clusters: {optimal_k}")
    return optimal_k, silhouette_scores

# Run clustering analysis
print("Finding optimal number of clusters...")
optimal_k, silhouette_scores = find_optimal_clusters_silhouette(formatted_dataset, max_k=20)

# Perform clustering with optimal k
print(f"\nPerforming clustering with k={optimal_k}")
kmeans_optimal = TimeSeriesKMeans(n_clusters=optimal_k, random_state=42)
cluster_labels = kmeans_optimal.fit_predict(formatted_dataset)

print(f"Cluster distribution:")
unique, counts = np.unique(cluster_labels, return_counts=True)
for cluster, count in zip(unique, counts):
    print(f"  Cluster {cluster}: {count} time series")


In [ ]:
# PYTHON APPROACHES FOR MULTIVARIATE TIME SERIES CLUSTERING

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from tslearn.clustering import TimeSeriesKMeans, KernelKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from dtaidistance import dtw
import seaborn as sns

# Generate sample multivariate time series data
np.random.seed(42)
n_series = 100
n_timepoints = 50
n_variables = 3

# Create synthetic data with different patterns
data = []
for i in range(n_series):
    if i < 33:  # Cluster 1: trending up
        ts = np.random.randn(n_timepoints, n_variables) + np.linspace(0, 2, n_timepoints)[:, np.newaxis]
    elif i < 66:  # Cluster 2: oscillating
        t = np.linspace(0, 4*np.pi, n_timepoints)
        ts = np.random.randn(n_timepoints, n_variables) + np.column_stack([np.sin(t), np.cos(t), np.sin(2*t)])
    else:  # Cluster 3: stationary
        ts = np.random.randn(n_timepoints, n_variables)
    
    data.append(ts)

data = np.array(data)
print(f"Data shape: {data.shape} (n_series, n_timepoints, n_variables)")

# METHOD 1: DTW-based clustering with tslearn
print("\n=== METHOD 1: DTW K-Means ===")

# Normalize the time series
scaler = TimeSeriesScalerMeanVariance()
data_scaled = scaler.fit_transform(data)

# DTW K-means clustering
dtw_kmeans = TimeSeriesKMeans(n_clusters=3, metric="dtw", random_state=42)
dtw_labels = dtw_kmeans.fit_predict(data_scaled)

print(f"DTW K-means silhouette score: {silhouette_score(data_scaled.reshape(n_series, -1), dtw_labels):.3f}")

# METHOD 2: Flatten and use traditional clustering
print("\n=== METHOD 2: Flattened Features + K-Means ===")

# Flatten multivariate time series
data_flat = data.reshape(n_series, -1)
scaler_flat = StandardScaler()
data_flat_scaled = scaler_flat.fit_transform(data_flat)

# K-means on flattened data
kmeans_flat = KMeans(n_clusters=3, random_state=42)
flat_labels = kmeans_flat.fit_predict(data_flat_scaled)

print(f"Flattened K-means silhouette score: {silhouette_score(data_flat_scaled, flat_labels):.3f}")

# METHOD 3: Feature extraction + clustering
print("\n=== METHOD 3: Feature Engineering + Clustering ===")

def extract_features(ts):
    """Extract statistical features from multivariate time series"""
    features = []
    for var in range(ts.shape[1]):
        series = ts[:, var]
        features.extend([
            np.mean(series),
            np.std(series),
            np.min(series),
            np.max(series),
            np.median(series),
            np.percentile(series, 25),
            np.percentile(series, 75),
            # Trend (slope of linear regression)
            np.polyfit(range(len(series)), series, 1)[0]
        ])
    return features

# Extract features for all time series
features = np.array([extract_features(ts) for ts in data])
features_scaled = StandardScaler().fit_transform(features)

# Clustering on features
kmeans_features = KMeans(n_clusters=3, random_state=42)
feature_labels = kmeans_features.fit_predict(features_scaled)

print(f"Feature-based K-means silhouette score: {silhouette_score(features_scaled, feature_labels):.3f}")

# METHOD 4: PCA + Clustering
print("\n=== METHOD 4: PCA Dimensionality Reduction + Clustering ===")

# Apply PCA to flattened data
pca = PCA(n_components=0.95)  # Keep 95% of variance
data_pca = pca.fit_transform(data_flat_scaled)

# K-means on PCA components
kmeans_pca = KMeans(n_clusters=3, random_state=42)
pca_labels = kmeans_pca.fit_predict(data_pca)

print(f"PCA + K-means silhouette score: {silhouette_score(data_pca, pca_labels):.3f}")
print(f"PCA components retained: {data_pca.shape[1]} (explaining {pca.explained_variance_ratio_.sum():.3f} of variance)")

# VISUALIZATION
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

methods = [
    ("True Labels", np.concatenate([np.zeros(33), np.ones(33), np.full(34, 2)])),
    ("DTW K-means", dtw_labels),
    ("Flattened K-means", flat_labels), 
    ("Feature K-means", feature_labels),
    ("PCA K-means", pca_labels)
]

# Plot first variable for each method
for idx, (method_name, labels) in enumerate(methods):
    if idx < 4:
        ax = axes[0, idx]
    else:
        ax = axes[1, 0]
    
    colors = ['red', 'blue', 'green', 'purple', 'orange']
    for i, ts in enumerate(data[:30]):  # Plot first 30 series
        ax.plot(ts[:, 0], color=colors[int(labels[i])], alpha=0.3)
    ax.set_title(f'{method_name}\n(Variable 1)')
    ax.set_xlabel('Time')

# Plot cluster comparison
axes[1, 1].axis('off')
comparison_df = pd.DataFrame({
    'True': np.concatenate([np.zeros(33), np.ones(33), np.full(34, 2)]),
    'DTW': dtw_labels,
    'Flattened': flat_labels,
    'Features': feature_labels,
    'PCA': pca_labels
})

# Confusion matrix between true and best method
from sklearn.metrics import adjusted_rand_score
scores = {}
for method in ['DTW', 'Flattened', 'Features', 'PCA']:
    scores[method] = adjusted_rand_score(comparison_df['True'], comparison_df[method])

print(f"\nAdjusted Rand Index (vs True Labels):")
for method, score in scores.items():
    print(f"{method}: {score:.3f}")

# Plot silhouette comparison
methods_scores = {
    'DTW': silhouette_score(data_scaled.reshape(n_series, -1), dtw_labels),
    'Flattened': silhouette_score(data_flat_scaled, flat_labels),
    'Features': silhouette_score(features_scaled, feature_labels),
    'PCA': silhouette_score(data_pca, pca_labels)
}

axes[1, 2].bar(methods_scores.keys(), methods_scores.values())
axes[1, 2].set_title('Silhouette Scores')
axes[1, 2].set_ylabel('Score')

# Feature importance plot (for feature-based method)
if len(features) > 0:
    feature_names = []
    for var in range(n_variables):
        feature_names.extend([f'V{var}_mean', f'V{var}_std', f'V{var}_min', f'V{var}_max', 
                            f'V{var}_median', f'V{var}_q25', f'V{var}_q75', f'V{var}_trend'])
    
    # PCA components of features to see importance
    pca_features = PCA(n_components=3)
    pca_features.fit(features_scaled)
    
    axes[1, 3].bar(range(len(pca_features.explained_variance_ratio_)), 
                   pca_features.explained_variance_ratio_)
    axes[1, 3].set_title('Feature PCA Explained Variance')
    axes[1, 3].set_xlabel('Component')

plt.tight_layout()
plt.show()

# RECOMMENDATIONS
print("\n=== RECOMMENDATIONS ===")
print("1. DTW K-means: Best for preserving temporal patterns and shape similarity")
print("2. Feature-based: Good interpretability, fast, works well with domain knowledge")
print("3. Flattened + PCA: Good compromise between speed and performance")
print("4. For large datasets: Use feature extraction or PCA first")
print("5. Consider hierarchical clustering for better cluster interpretability")

In [ ]:
data




In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Example usage
# Create a test directed graph
G = nx.DiGraph()
G.add_edges_from([
    (1, 2, {'weight': 3}),
    (2, 3, {'weight': 5}),
    (3, 4, {'weight': 2}),
    (4, 1, {'weight': 4}),
])

In [ ]:

from huggingface_hub import notebook_login
#notebook_login()

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import logging
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)


# Example text
text1 = "Renewable energy is the future."
text2 = "However, it cannot fully replace fossil fuels due to reliability issues."
text1 = "Earth is not flat"
text2 = "Der Hahn kräht"
text2 = "Atomar power is not safe"

# Detect fake news
model_name = "openai-community/roberta-base-openai-detector"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)
result = pipe(text1)
print(result)

# Relation classification model
relation_pipeline = pipeline("text-classification", model="raruidol/ArgumentMining-EN-ARI-AIF-RoBERTa_L")
result = relation_pipeline(f"{text1} [SEP] {text2}")
print(result)


In [ ]:
#!pip install spacy
#!pip install textdescriptives
#!spacy download en_core_web_sm
import spacy
import textdescriptives as td
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textdescriptives/quality")
doc = nlp("The world is changed. I feel it in the \n water. I feel it in\n the earth. I smell it in the air. Much that once was is lost, for none now live who remember it.")

# all attributes are stored as a dict in the ._.quality attribute
doc._.quality
# check if the document passed all quality checks
doc._.passed_quality_check

# extract to dataframe
td.extract_df(doc)

In [ ]:

import re
import json
expected_fields=["study_subject", "status", "comment"]
llm_message = '{"status": "welt", "comment":"nonnne", "study_subject": "math"}'
regex = r"{[\s\S]+}"
print(re.search(regex, llm_message))
json_string = re.search(regex, llm_message).group()
json_output = json.loads(json_string)
for field in expected_fields:
    if not json_output[field]:
        continue
json_valid = True
print(json_output)
